In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub

import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, HfArgumentParser, pipeline, AutoConfig, GPTNeoXForCausalLM
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import pipeline, TextGenerationPipeline

/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-29 07:47:47.960959: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 07:47:48.747702: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
#model_path = '/scratch1/jhoff/checkpoints/generator_pythia-1B/runs/step_8000_merged'
#model_path = "EleutherAI/pythia-1B-Deduped"
#model_path = '/scratch1/jhoff/checkpoints/finetuned-pythia-6.9B-deduped/checkpoint_5000_merged'
model_path = '/scratch1/jhoff/checkpoints/finetuned-pythia-6.9B-deduped/checkpoint_25000_merged'

model = GPTNeoXForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.config.pad_token_id = model.config.eos_token_id
model = model.cuda().eval()

Loading checkpoint shards: 100%|██████████| 3/3 [02:00<00:00, 40.15s/it]


In [6]:
pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, device='cuda:0')
pipeline

In [8]:
generation_kwargs = {
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.pad_token_id,
    # "eos_token_id": 100_000,
    "min_length": 32,
    "max_length": 128,
}

questions = [
    "What is a^2 + b^2 = c^2?",
    'How does the rule of pythagoras work?',
    "Who is the president of the U.S.?",
    # "Why is the sky blue?", 
    # "Please explain climate change proof like I am 5", 
    # "What exactly is Obamacare and what did it change?", 
    'The 3:1 microphone rule', 
    
    "why are humans considered three dimensional beings when we also move through the fourth dimension of time?\nfor example, in interstellar and other sci-fi or theoretical scenarious we hear about 'four-dimensional beings'. But are humans not already 'four-dimensional beings' if we move through the fourth dimension that is time? ", 
    'Would there be any legal ramifications if a major company were to use an idea from a website like Reddit?\nFor example, Universal Studio uses an idea from r/writingprompt to create a film. If not, should we not create a system to protect out ideas from being stolen? ', 
    "What's the difference between blu-ray quality and 1080p? \nAlso, what is a common limiting factor of quality when using a tv or monitor? ", 
    'Why does blood taste like metal? ', 
    'Why do all of the new hires at my job suck? ', 
    'Best yo mama jokes please? ', 
    "Why can people sue Ashley Madison even though they weren't the ones that released the information? ", 
    'Toyota can\'t make an exact copy of a Porsche 911 bodywork, or a Ferrari, put a Toyota engine and sticker on it and then sell it. However, they can design their cars to be "sportier", as a selling point. Where/how is the line drawn between inspiring, and copying? ', 
]      

for question in questions:
    result = pipeline(f"""Question: {question}\n\n\nAnswer:""", **generation_kwargs)
    result = result[0]['generated_text']
    print('-' * 30)
    print(f'{result}')
    print('-')
    print(f"Output length: {len(result)}")
    print(f"Output tokens: {tokenizer(result)['input_ids'].__len__()}")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


------------------------------
Question: What is a^2 + b^2 = c^2?


Answer: We assume that c is real. We compute:

	b^2 - 4a = c^2

	b^2(1-4a ) = c^2

	(b^2 - 4a)(1-4a) = c^2

	b^4 - 8a^2 + 4a^2 = c^2

	b^4(1-8a^2) - 8a^2(1-4a^2) = c^2

	b
-
Output length: 208
Output tokens: 128


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


------------------------------
Question: How does the rule of pythagoras work?


Answer: Pythagoras was the farnsworth haim i/
  j concept and the simple livvers are) K -S
  haev we hecrrled and jodiided in order to
  finally acquire a simple life. dua 13 (e)
  
  
End of meetings.


Notes:


With this story as a bed, I've been filling...

<!----------------------------->
<html>
<head>
<!-- Webfoot Auto-VBlank Head
-
Output length: 387
Output tokens: 128


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

------------------------------
Question: Why do all of the new hires at my job suck? 


Answer: Because they’re all idiots who don’t have sharp edges.Bill Maher – Boxers or Briefs

God bless Bill Maher for saying what so many of us are thinking out in the world and offering a plan of attack to those in power who have allowed this war of class warfare in the country today.

“... Why is quadrennial warfare between the rich upper 10 percent of the population and the poor lower 10 percent of our population more important than our national security?... The script
-
Output length: 533
Output tokens: 124


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


------------------------------
Question: Best yo mama jokes please? 


Answer: Mary: What you sound like in the process of having sex? Francisco: Macarena.



There are some viral tendencies that aren't because of the application - if someone is on the network and logging on and staying connected, people will just see a flurry of actions by their friends and family. Then it boils down to those people's moms, those individuals wanting to spread the word that these are actually popular.

With numbers like that of poker mama jokes and more-recent things, like Barack Obama's "you didn't build and (
-
Output length: 570
Output tokens: 128


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


------------------------------
Question: Why can people sue Ashley Madison even though they weren't the ones that released the information? 


Answer: Because of the doctrine of vicarious liability, which states that a person can be held responsible for the actions of third persons who do not even have to be a party to a lawsuit. The original whistleblower lawsuit could relate to allegedly unlawful conduct by other users, and therefore, the plaintiffs are seeking damages on behalf of anyone who was victimized by the company's use of the ‘hide my details’ feature.


Question: Was any form of compensation offered to the data that was leaked?


Answer: No
-
Output length: 629
Output tokens: 128
------------------------------
Question: Toyota can't make an exact copy of a Porsche 911 bodywork, or a Ferrari, put a Toyota engine and sticker on it and then sell it. However, they can design their cars to be "sportier", as a selling point. Where/how is the line drawn between inspiring, and copy